In [1]:
#need to download GoogleNews-vectors-negative300.bin first
from gensim.models import Word2Vec, KeyedVectors
#use your path!!!
pathToGoogleNews300 = '..//fromBlogOfShaneLynnWordEmbeddingsWithSpacyAndGensim_GoogleNews300//data//GoogleNews-vectors-negative300.bin'
model = KeyedVectors.load_word2vec_format(pathToGoogleNews300, binary=True)

In [2]:
import pandas as pd
#need to download Gungor_2018_VictorianAuthorAttribution_data-train.csv from http://archive.ics.uci.edu/ml/datasets/Victorian+Era+Authorship+Attribution
#use your path!!!
pathToGungorVict = '..//..//DS7004//u1720146_DS7004_courseworkCodeAndData//preparationWorks//fromDS7003_Gungor2018VictorianAuthorAttribution_NGram//Gungor_2018_VictorianAuthorAttribution_data-train.csv'
gungorVictRow = pd.read_csv(pathToGungorVict, encoding = 'ISO-8859-1')
#48: Washington Irving/ 13: Frances Hodgson Burnett/ 22: Jacob Abbott/ 25: James Payn/ 36: Oliver Optic

In [3]:
for i in [13, 22, 25, 36, 48]:
    allLines = gungorVictRow.loc[gungorVictRow['author'] == i]
    lines350 = allLines.iloc[0:350]
    linesLast20 = allLines.iloc[-20:]
    try:
        train = train.append(lines350)
        test = test.append(linesLast20)
    except:
        train = lines350
        test = linesLast20
train = train.sample(frac=1, random_state=42).reset_index(drop = True) #1750 lines suffled
test = test.sample(frac=1, random_state=42).reset_index(drop = True) #100 lines suffledtest = authorsLines.sample(frac=0.3, random_state=42)

In [13]:
model.most_similar(positive=['king', 'woman'], negative=['man'])

[('queen', 0.7118192911148071),
 ('monarch', 0.6189674139022827),
 ('princess', 0.5902431607246399),
 ('crown_prince', 0.5499460697174072),
 ('prince', 0.5377321243286133),
 ('kings', 0.5236844420433044),
 ('Queen_Consort', 0.5235945582389832),
 ('queens', 0.5181134343147278),
 ('sultan', 0.5098593235015869),
 ('monarchy', 0.5087411999702454)]

In [14]:
model.most_similar(positive=['better', 'bad'], negative=['good'])

[('worse', 0.7288720607757568),
 ('uglier', 0.5725537538528442),
 ('sooner', 0.5309501886367798),
 ('dumber', 0.522783637046814),
 ('differently', 0.5173395872116089),
 ('quicker', 0.5150286555290222),
 ('stupider', 0.5140471458435059),
 ('harsher', 0.5098250508308411),
 ('harder', 0.5080496072769165),
 ('scarier', 0.5044300556182861)]

In [16]:
model.most_similar(positive=['lead', 'saw'], negative=['led'])

[('see', 0.4809260070323944),
 ('looked', 0.45732152462005615),
 ('seeing', 0.4199181795120239),
 ('advantage', 0.40815281867980957),
 ('knew', 0.3928772211074829),
 ('thought', 0.38903290033340454),
 ('noticed', 0.38654080033302307),
 ('squandered_glorious', 0.3810598850250244),
 ('midway_through', 0.37417805194854736),
 ('chances', 0.36055076122283936)]

In [15]:
model.doesnt_match("man woman child kitchen".split())

C:\Users\Eric\anaconda3\lib\site-packages\gensim\models\keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'kitchen'

In [4]:
import gensim
all_stopwords = set(gensim.parsing.preprocessing.STOPWORDS)

In [5]:
#be careful: nword and counter must be integers --Chiu
import numpy as np  # Make sure that numpy is imported

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    index2word_set2 = all_stopwords
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set:
            if word in index2word_set2: 
                nwords = nwords + 1
                featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    if nwords == 0:
        nwords = 1
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
        if counter%50 == 0:
            haha = counter; hihi = len(reviews)
            print(f"Review {haha} of {hihi}") #% (counter, len(reviews))
       # 
       # Call the function (defined above) that makes average feature vectors
        #reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
       #
       # Increment the counter
        counter = counter + 1
    return reviewFeatureVecs

In [6]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [7]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.
num_features = 300
clean_train_reviews = []
for review in train["text"]:
    #clean_train_reviews.append( review_to_wordlist( review, \
        #remove_stopwords=True ))
    clean_train_reviews.append( review_to_wordlist( review ))

trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )

print("Creating average feature vecs for test reviews")
clean_test_reviews = []
for review in test["text"]:
    #clean_test_reviews.append( review_to_wordlist( review, remove_stopwords=True ))
    clean_test_reviews.append( review_to_wordlist( review ))

testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )

Review 0 of 1750


<ipython-input-5-206be6dc17b3>:15: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  index2word_set = set(model.wv.index2word)


Review 50 of 1750
Review 100 of 1750
Review 150 of 1750
Review 200 of 1750
Review 250 of 1750
Review 300 of 1750
Review 350 of 1750
Review 400 of 1750
Review 450 of 1750
Review 500 of 1750
Review 550 of 1750
Review 600 of 1750
Review 650 of 1750
Review 700 of 1750
Review 750 of 1750
Review 800 of 1750
Review 850 of 1750
Review 900 of 1750
Review 950 of 1750
Review 1000 of 1750
Review 1050 of 1750
Review 1100 of 1750
Review 1150 of 1750
Review 1200 of 1750
Review 1250 of 1750
Review 1300 of 1750
Review 1350 of 1750
Review 1400 of 1750
Review 1450 of 1750
Review 1500 of 1750
Review 1550 of 1750
Review 1600 of 1750
Review 1650 of 1750
Review 1700 of 1750
Creating average feature vecs for test reviews
Review 0 of 100
Review 50 of 100


In [8]:
# Fit a random forest to the training data, using 100 trees
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier( n_estimators = 100 )

print("Fitting a random forest to labeled training data...")
forest = forest.fit( trainDataVecs, train["author"] )

# Test & extract results 
result = forest.predict( testDataVecs )

# Write the test results 
output = pd.DataFrame( data={"true_author":test["author"], "pred_author":result} )
output.to_csv( "Word2Vec_AverageVectors.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...


In [9]:
confusion_matrix = pd.crosstab(output['true_author'], output['pred_author'], rownames=['Actural'], colnames=['Predicted'])
print(confusion_matrix)

Predicted  13  22  25  36  48
Actural                      
13         12   1   2   3   2
22          0  20   0   0   0
25          4   0  16   0   0
36          0   2   0  17   1
48          0   6   0   6   8


In [10]:
from sklearn import metrics
print('Accuracy: ', metrics.accuracy_score(output['true_author'], output['pred_author']))

Accuracy:  0.73


In [11]:
print('f1 score: ', metrics.f1_score(output['true_author'], output['pred_author'], average = 'weighted'))

f1 score:  0.716071585495496
